# Playing Around with Traffic Data

Hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN.
Hourly weather features and holidays included for impacts on traffic volume

holiday: Categorical US National holidays plus regional holiday, Minnesota State Fair 
temp: Numeric Average temp in kelvin 
rain_1h: Numeric Amount in mm of rain that occurred in the hour 
snow_1h: Numeric Amount in mm of snow that occurred in the hour 
clouds_all: Numeric Percentage of cloud cover 
weather_main: Categorical Short textual description of the current weather 
weather_description: Categorical Longer textual description of the current weather 
date_time: DateTime Hour of the data collected in local CST time 
traffic_volume: Numeric Hourly I-94 ATR 301 reported westbound traffic volume

Data obtained from UCI Machine Learning Repository

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
df = pd.read_csv('Metro_Interstate_Traffic_Volume_data.csv')
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [3]:
#check data type of date_time
type(df['date_time'][3])

#convert date_time from string to timestamp data type
df['date_time'] = pd.to_datetime(df['date_time'])
type(df['date_time'][3])

#check for missing dates
import datetime as dt
df['date'] = df['date_time'].dt.date
df['date'].value_counts()

2012-12-16    81
2018-04-14    68
2017-05-01    68
2012-12-10    67
2015-07-06    64
2018-09-20    64
2017-01-21    62
2016-11-30    61
2016-12-25    61
2018-09-04    61
2018-08-24    61
2018-04-15    60
2017-10-27    60
2013-04-19    60
2016-04-24    60
2017-08-26    58
2018-02-24    58
2017-01-20    57
2017-04-15    57
2017-05-17    57
2018-09-19    56
2014-05-08    56
2013-12-03    56
2016-11-18    56
2018-06-19    55
2017-05-21    55
2017-04-26    55
2016-11-27    55
2015-08-19    54
2012-10-26    54
              ..
2014-06-13     6
2013-08-27     6
2014-06-20     6
2014-06-12     6
2013-10-24     6
2014-06-29     5
2013-09-03     4
2014-06-25     4
2014-06-24     4
2014-06-21     3
2013-10-02     3
2015-06-12     3
2014-07-24     3
2014-06-27     3
2014-06-22     3
2014-06-16     2
2015-06-13     2
2015-06-22     2
2014-08-08     2
2013-10-01     2
2014-06-30     2
2014-06-17     2
2015-06-20     1
2013-08-31     1
2013-10-27     1
2013-01-29     1
2013-09-01     1
2015-06-14    

There are 2189 days in the date range between the first date in the dataset and the last, so there are 329 missing days! Some days also have only one entry, meaning a lot of missing hours.

Because of this missing data, it will be necessary to not treat this as time series data.

Some days have more than 24 entries--- investigate that next. Particularly Dec. 16, 2012, since that has 81 entries.

In [4]:
df[df['date'] == pd.to_datetime('2012-12-16').date()]

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date
2036,None,276.06,0.0,0.0,90,Clouds,overcast clouds,2012-12-16 00:00:00,1098,2012-12-16
2037,None,276.25,0.0,0.0,90,Clouds,overcast clouds,2012-12-16 01:00:00,704,2012-12-16
2038,None,276.28,0.0,0.0,90,Clouds,overcast clouds,2012-12-16 02:00:00,588,2012-12-16
2039,None,275.71,0.0,0.0,90,Mist,mist,2012-12-16 03:00:00,307,2012-12-16
2040,None,275.71,0.0,0.0,90,Rain,light rain,2012-12-16 03:00:00,307,2012-12-16
2041,None,275.71,0.0,0.0,90,Haze,haze,2012-12-16 03:00:00,307,2012-12-16
2042,None,275.71,0.0,0.0,90,Drizzle,light intensity drizzle,2012-12-16 03:00:00,307,2012-12-16
2043,None,275.34,0.0,0.0,90,Mist,mist,2012-12-16 04:00:00,271,2012-12-16
2044,None,275.34,0.0,0.0,90,Rain,light rain,2012-12-16 04:00:00,271,2012-12-16
2045,None,275.34,0.0,0.0,90,Drizzle,light intensity drizzle,2012-12-16 04:00:00,271,2012-12-16


Hourly entry seems consistent. Temperature doesn't seem to change within the hour. Even traffic volume "stays the same." 
Multiple entries per hour seem to occur because of changes in weather during that hour.

Because traffic volume is measured hourly, having multiple entries for each hour biases any analysis of determinants of traffic volume to show less of an effect from independent variables that change between these multiple entries.

In [5]:
#Start by dropping all duplicates, in case there are any
df.drop_duplicates(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description', 'date_time', 'traffic_volume'], inplace=True)
#df['date_time'].value_counts()
#df[df['date'] == pd.to_datetime('2016-05-26').date()]
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02


In [6]:
#Mark all duplicate-timestamped rows first with a new column with a 1 if that row is part of a duplicate set
df['weather_main_mark'] = np.nan
df.loc[0, 'weather_main_mark'] = 0

for m in range(1, len(df)-2):
    if df.loc[m,'date_time'] == df.loc[m-1,'date_time']:
        df.loc[m, 'weather_main_mark'] = 1
    elif df.loc[m,'date_time'] == df.loc[m+1,'date_time']:
        df.loc[m, 'weather_main_mark'] = 1
    else:
        df.loc[m, 'weather_main_mark'] = 0

df['weather_main_mark'].fillna(value=0, inplace=True)

In [7]:
df['weather_main'].value_counts()

Clouds          15158
Clear           13384
Mist             5949
Rain             5672
Snow             2875
Drizzle          1820
Haze             1360
Thunderstorm     1033
Fog               912
Smoke              20
Squall              4
Name: weather_main, dtype: int64

To get rid of duplicate cells with the same timestamp, there must be a scheme for reconciling conflicting weather descriptions. Unfortunately, the data do not convey how long each weather description lasted within a given hour (e.g., in an hour with both rain and clearness, we cannot determine how long it rained for).

As a general principle, the more "severe" of two conflicting weather phenomena will triumph. For example, in an hour with both fog and mist, that hour's weather will be marked as "Fog" because fog, by definition, limits visibility more than mist does.

For example, since rain must involve clouds, two rows from the same hour with weather "Rain" and "Clouds" will be consolidated into one rows with the weather description "Rain."

By consolidating rows in this way, we are changing what it means when a row says "Rain," "Snow," or so forth. We can no longer assume that the rain or snow occurred for the entirety of the hour. In terms of the effect of weather on traffic, this effect can benefit the model in cases where rain or snow occurs for only part of the hour because the ground stays wet or snowy after rain or snow stops falling. Unfortunately, because of the missing data, it does not seem possible to apply this reasoning to previous hours (e.g., by adding a variable to each row that conveys whether rain fell within the past 6 hours).

Of course, this principle does not address all conflicts. Severity of weather is subjective to some extent.

In [8]:
#create new dataframe of only duplicate rows on weather

df_dup = df[df['weather_main_mark'] == 1].copy(deep=True)
df_dup.reset_index(inplace=True)
df_dup.drop(labels = 'index', axis=1, inplace=True)
df_dup['weather_main_2'] = np.nan
df_dup.loc[0, 'weather_main_2'] = df_dup.loc[0, 'weather_main']

for m in range(2, len(df_dup)-2):
    #2-duplicate scenario
    if ((df_dup.loc[m,'date_time'] == df_dup.loc[m+1,'date_time'])&(df_dup.loc[m,'date_time'] != df_dup.loc[m-1,'date_time']))&(df_dup.loc[m,'date_time'] != df_dup.loc[m+2,'date_time']):
        if df_dup.loc[m:m+1, 'weather_main'].isin(['Drizzle', 'Rain']).sum()>1:
            if df_dup.loc[m:m+1, 'weather_description'].isin(['light rain']).sum()>0:
                df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
            else:
                df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif (df_dup.loc[m:m+1, 'weather_main'].isin(['Rain', 'Snow']).sum()>1)|(df_dup.loc[m:m+1, 'weather_main'].isin(['Drizzle', 'Snow']).sum()>1):
            df_dup.loc[m, 'weather_main_2'] = 'Mixed Precipitation'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Rain', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Drizzle', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Rain', 'Clouds']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        #mist interactions
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Mist', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Mist', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Drizzle', 'Mist']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Mist', 'Haze']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Misty Haze'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Mist', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        #haze interactions
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Drizzle']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Smoke']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Smoke'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Haze', 'Clouds']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Haze'
        #fog interactions
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Drizzle']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Mist']).sum()>1) | (df_dup.loc[m:m+1, 'weather_main'].isin(['Fog', 'Haze']).sum()>1):
            df_dup.loc[m, 'weather_main_2'] = 'Fog'
        #other interactions
        elif df_dup.loc[m:m+1, 'weather_main'].isin(['Clouds', 'Clear']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Clouds'
        else:
            df_dup.loc[m, 'weather_main_2'] = 1
    #Now the other half...
    elif ((df_dup.loc[m,'date_time'] == df_dup.loc[m-1,'date_time'])&(df_dup.loc[m,'date_time'] != df_dup.loc[m+1,'date_time']))&(df_dup.loc[m,'date_time'] != df_dup.loc[m-2,'date_time']):
        if df_dup.loc[m-1:m, 'weather_main'].isin(['Drizzle', 'Rain']).sum()>1:
            if df_dup.loc[m-1:m, 'weather_description'].isin(['light rain']).sum()>0:
                df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
            else:
                df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif (df_dup.loc[m-1:m, 'weather_main'].isin(['Rain', 'Snow']).sum()>1)|(df_dup.loc[m-1:m, 'weather_main'].isin(['Drizzle', 'Snow']).sum()>1):
            df_dup.loc[m, 'weather_main_2'] = 'Mixed Precipitation'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Rain', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Drizzle', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Rain', 'Clouds']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        #mist interactions
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Mist', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Mist', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Drizzle', 'Mist']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Mist', 'Haze']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Misty Haze'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Mist', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        #haze interactions
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Drizzle']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Smoke']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Smoke'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Haze', 'Clouds']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Haze'
        #fog interactions
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Rain']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Rain'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Snow']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Snow'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Drizzle']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Drizzle'
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Thunderstorm']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Mist']).sum()>1)|(df_dup.loc[m-1:m, 'weather_main'].isin(['Fog', 'Haze']).sum()>1):
            df_dup.loc[m, 'weather_main_2'] = 'Fog'
        #other interactions
        elif df_dup.loc[m-1:m, 'weather_main'].isin(['Clouds', 'Clear']).sum()>1:
            df_dup.loc[m, 'weather_main_2'] = 'Clouds'
        else:
            df_dup.loc[m, 'weather_main_2'] = 1
    else:
        df_dup.loc[m, 'weather_main_2'] = df_dup.loc[m, 'weather_main']
len(df_dup)

13042

In [9]:
#>2-duplicate scenario
for m in range(2, len(df_dup)-2):
    #all except first and last
    #3 instances
    if (((df_dup.loc[m,'date_time'] == df_dup.loc[m+1,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-1,'date_time'])))&((df_dup.loc[m,'date_time'] != df_dup.loc[m+2,'date_time'])&(df_dup.loc[m,'date_time'] != df_dup.loc[m-2,'date_time'])):
        if (df_dup.loc[m-1:m+1, 'weather_main'].isin(['Thunderstorm']).sum()>1):
            df_dup.loc[m-1:m+1, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m-1:m+1, 'weather_main'].isin(['Rain', 'Snow', 'Squall']).sum()>1)|(df_dup.loc[m-1:m+1, 'weather_main'].isin(['Drizzle', 'Snow', 'Squall']).sum()>1):
            df_dup.loc[m-1:m+1, 'weather_main_2'] = 'Mixed Precipitation'
        else:
            df_dup.loc[m-1:m+1, 'weather_main_2'] = df_dup.loc[m, 'weather_main_2']
    #4 instances - 2nd one
    elif (((df_dup.loc[m,'date_time'] == df_dup.loc[m+1,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-1,'date_time'])))&((df_dup.loc[m,'date_time'] == df_dup.loc[m+2,'date_time'])&(df_dup.loc[m,'date_time'] != df_dup.loc[m-2,'date_time'])):
        if (df_dup.loc[m-1:m+2, 'weather_main'].isin(['Thunderstorm']).sum()>0):
            df_dup.loc[m-1:m+2, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m-1:m+2, 'weather_main'].isin(['Rain', 'Snow', 'Squall']).sum()>1)|(df_dup.loc[m-1:m+2, 'weather_main'].isin(['Drizzle', 'Snow', 'Squall']).sum()>1):
            df_dup.loc[m-1:m+2, 'weather_main_2'] = 'Mixed Precipitation'
        else:
            df_dup.loc[m-1:m+2, 'weather_main_2'] = df_dup.loc[m, 'weather_main_2']
    #4 instances - 3rd one
    elif (((df_dup.loc[m,'date_time'] == df_dup.loc[m+1,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-1,'date_time'])))&((df_dup.loc[m,'date_time'] != df_dup.loc[m+2,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-2,'date_time'])):
        if (df_dup.loc[m-2:m+1, 'weather_main'].isin(['Thunderstorm']).sum()>0):
            df_dup.loc[m-2:m+1, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m-2:m+1, 'weather_main'].isin(['Rain', 'Snow', 'Squall']).sum()>1)|(df_dup.loc[m-2:m+1, 'weather_main'].isin(['Drizzle', 'Snow', 'Squall']).sum()>1):
            df_dup.loc[m-2:m+1, 'weather_main_2'] = 'Mixed Precipitation'
        else:
            df_dup.loc[m-2:m+1, 'weather_main_2'] = df_dup.loc[m, 'weather_main_2']
    #5-6 instances
    elif (((df_dup.loc[m,'date_time'] == df_dup.loc[m+1,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-1,'date_time'])))&((df_dup.loc[m,'date_time'] == df_dup.loc[m+2,'date_time'])&(df_dup.loc[m,'date_time'] == df_dup.loc[m-2,'date_time'])):
        if (df_dup.loc[m-2:m+2, 'weather_main'].isin(['Thunderstorm']).sum()>0):
            df_dup.loc[m-2:m+2, 'weather_main_2'] = 'Thunderstorm'
        elif (df_dup.loc[m:m+2, 'weather_main'].isin(['Rain', 'Snow', 'Squall']).sum()>1)|(df_dup.loc[m-2:m+2, 'weather_main'].isin(['Drizzle', 'Snow', 'Squall']).sum()>1):
            df_dup.loc[m-2:m+2, 'weather_main_2'] = 'Mixed Precipitation'
        else:
            df_dup.loc[m-2:m+2, 'weather_main_2'] = df_dup.loc[m, 'weather_main_2']
    else:
        df_dup.loc[m, 'weather_main_2'] = df_dup.loc[m, 'weather_main_2']
        

In [10]:
df_dup.drop_duplicates(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main_2', 'date_time', 'traffic_volume'], inplace=True)
df_dup['date_time'].value_counts()
df_dup['weather_main_2'].fillna(method='ffill', inplace=True)
len(df_dup)

5510

In [11]:
df_dup.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2
0,None,281.25,0.0,0.0,99,Rain,light rain,2012-10-10 07:00:00,6793,2012-10-10,1.0,Rain
1,None,281.25,0.0,0.0,99,Drizzle,light intensity drizzle,2012-10-10 07:00:00,6793,2012-10-10,1.0,Rain
2,None,280.10,0.0,0.0,99,Rain,light rain,2012-10-10 08:00:00,6283,2012-10-10,1.0,Drizzle
4,None,279.61,0.0,0.0,99,Rain,light rain,2012-10-10 09:00:00,5680,2012-10-10,1.0,Drizzle
6,None,282.43,0.0,0.0,57,Drizzle,light intensity drizzle,2012-10-14 09:00:00,2685,2012-10-14,1.0,Mist


In [12]:
#append the formerly duplicate rows to the working copy
df.drop(labels= df[df['weather_main_mark'] == 1].index, axis=0, inplace=True)
df['weather_main_2'] = df['weather_main']
df = df.append(df_dup, ignore_index=True)
len(df)
#df['weather_main_2'].value_counts()

40655

Length recorded at 40655 rows.

Now, we will check to see if there are still rows that repeat timestamps.

In [13]:
df['date_time'].value_counts()

2016-12-07 17:00:00    2
2016-07-12 14:00:00    2
2014-01-24 16:00:00    2
2014-01-03 20:00:00    2
2016-05-26 10:00:00    2
2016-12-16 08:00:00    2
2014-01-15 00:00:00    2
2014-01-19 16:00:00    2
2016-05-26 14:00:00    2
2016-12-04 09:00:00    2
2014-05-06 12:00:00    2
2016-05-27 14:00:00    2
2016-06-01 18:00:00    2
2016-06-01 14:00:00    2
2014-02-25 01:00:00    2
2014-01-17 16:00:00    2
2016-05-27 18:00:00    2
2016-05-25 18:00:00    2
2016-05-30 10:00:00    2
2016-05-28 18:00:00    2
2016-05-25 10:00:00    2
2016-07-13 18:00:00    2
2016-12-05 16:00:00    2
2014-02-25 03:00:00    2
2016-12-18 14:00:00    2
2016-05-31 14:00:00    2
2016-05-29 10:00:00    2
2016-09-25 20:00:00    2
2016-06-02 14:00:00    2
2016-05-24 18:00:00    2
                      ..
2016-08-12 08:00:00    1
2016-11-17 05:00:00    1
2018-07-08 08:00:00    1
2014-08-06 08:00:00    1
2016-11-05 21:00:00    1
2018-01-29 21:00:00    1
2017-07-28 00:00:00    1
2014-02-23 00:00:00    1
2013-01-02 00:00:00    1


There are still repeated timestamps, but no timestamp appears more than twice. What was happening at 5 pm on December 7, 2016, that wasn't necessarily a change in weather?

In [14]:
df[df['date_time'] =='2016-12-07 17:00:00']

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2
38153,None,265.75,0.0,0.0,90,Snow,light snow,2016-12-07 17:00:00,6143,2016-12-07,1.0,Mixed Precipitation
38154,None,265.96,0.0,0.0,90,Snow,light snow,2016-12-07 17:00:00,6143,2016-12-07,1.0,Mixed Precipitation


Temperatures are different within the hour. So we need a new column to convey the average temperature within the hour. Because each timestamp is repeated once at most, the code to create this new column can divide the sum of all (two) temperatures within the hour by 2.

Because <i>any</i> new column can assume only 2 cells, we can write a function that fills in a column with the average of two rows with identical timestamps, and copies over data otherwise.

The function takes a dataframe and three of its columns as inputs. When two contiguous rows in the dataframe have the same value in one column, the function averages the the two values of a targeted second column and fills in a third column with these averages. Otherwise, it simply copies over the value from the second column to the third.

In [15]:
df['temp_avg'] = np.nan
df.loc[0, 'temp_avg'] = df.loc[0, 'temp']
df.reset_index(inplace=True)
df.drop(labels = 'index', axis=1, inplace=True)

def average_duplicates(df, dup, s, f):
    for c in range(1, len(s)-1):
        if df.loc[c, dup.name] == df.loc[c-1, dup.name]:
            df.loc[c, s.name] = (df.loc[c, s.name]+df.loc[c-1, s.name])/2
        elif df.loc[c, dup.name] == df.loc[c+1, dup.name]:
            df.loc[c, s.name] = (df.loc[c, s.name]+df.loc[c+1, s.name])/2
        else:
            df.loc[c, s.name] = df.loc[c, f.name]

average_duplicates(df, df['date_time'], df['temp_avg'], df['temp'])

#fill in missing values
df['temp_avg'].fillna(value=df['temp'], inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2,temp_avg
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02,0.0,Clouds,288.28
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02,0.0,Clouds,289.36
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02,0.0,Clouds,289.58
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02,0.0,Clouds,290.13
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02,0.0,Clouds,291.14


In [16]:
#drop duplicates again and check for more
df.drop_duplicates(['holiday', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main_2', 'date_time', 'traffic_volume'], inplace=True)
len(df)

40611

Length recorded at 40611 rows.

In [17]:
df['date_time'].value_counts()

2014-02-25 01:00:00    2
2014-07-06 07:00:00    2
2014-01-11 16:00:00    2
2014-01-17 03:00:00    2
2014-04-21 16:00:00    2
2016-10-08 17:00:00    2
2016-05-31 18:00:00    2
2014-06-01 05:00:00    2
2016-05-27 10:00:00    2
2014-01-16 16:00:00    2
2016-09-25 20:00:00    2
2016-05-24 18:00:00    2
2014-01-17 12:00:00    2
2016-06-02 14:00:00    2
2016-06-29 20:00:00    2
2014-01-09 16:00:00    2
2014-05-31 18:00:00    2
2016-05-28 18:00:00    2
2016-05-25 10:00:00    2
2014-01-20 16:00:00    2
2014-05-06 12:00:00    2
2016-05-29 14:00:00    2
2016-12-05 16:00:00    2
2014-01-17 16:00:00    2
2014-05-10 04:00:00    2
2014-01-24 16:00:00    2
2014-01-15 00:00:00    2
2014-01-19 16:00:00    2
2016-05-28 14:00:00    2
2016-12-06 14:00:00    2
                      ..
2017-03-18 21:00:00    1
2018-07-25 21:00:00    1
2014-07-21 19:00:00    1
2015-10-02 05:00:00    1
2017-07-10 02:00:00    1
2014-02-05 02:00:00    1
2012-12-15 02:00:00    1
2014-03-04 10:00:00    1
2016-08-12 08:00:00    1


There are still more duplicates. Once again, I will look at the first timestamp to pop up.

In [18]:
df[df['date_time'] == '2014-02-25 01:00:00']

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2,temp_avg
36404,None,255.78,0.0,0.0,40,Clouds,scattered clouds,2014-02-25 01:00:00,293,2014-02-25,1.0,Clouds,255.78
36405,None,255.32,0.0,0.0,1,Clear,sky is clear,2014-02-25 01:00:00,293,2014-02-25,1.0,Clouds,255.32


This duplication remained because of a change in cloud cover. We can apply the same principle used with temperature to cloud cover.

In [19]:
#for duplication because of change in cloud cover, average cloud cover stats
df.reset_index(inplace=True)
df.drop(labels = 'index', axis=1, inplace=True)
df['cloud_cover_avg'] = np.nan
df.loc[0, 'cloud_cover_avg'] = df.loc[0, 'clouds_all']
average_duplicates(df, df['date_time'], df['cloud_cover_avg'], df['clouds_all'])

        
#fill in missing values
df['cloud_cover_avg'].fillna(value=df['clouds_all'], inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2,temp_avg,cloud_cover_avg
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02,0.0,Clouds,288.28,40.0
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02,0.0,Clouds,289.36,75.0
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02,0.0,Clouds,289.58,90.0
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02,0.0,Clouds,290.13,90.0
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02,0.0,Clouds,291.14,75.0


In [20]:
df.drop_duplicates(['holiday', 'rain_1h', 'snow_1h', 'weather_main_2', 'date_time', 'traffic_volume'], inplace=True)
len(df)

40584

Length recorded at 40584 rows.

In [21]:
df['date_time'].value_counts()

2016-09-25 20:00:00    2
2014-01-19 16:00:00    2
2014-06-01 05:00:00    2
2014-07-06 07:00:00    2
2016-05-25 10:00:00    2
2014-05-31 18:00:00    2
2016-05-28 14:00:00    2
2014-01-24 16:00:00    2
2016-05-27 10:00:00    2
2018-01-05 13:00:00    1
2016-04-11 02:00:00    1
2013-12-29 02:00:00    1
2017-04-07 06:00:00    1
2017-05-08 09:00:00    1
2015-09-21 23:00:00    1
2014-08-03 13:00:00    1
2013-06-12 13:00:00    1
2018-09-30 06:00:00    1
2016-12-22 01:00:00    1
2013-06-07 07:00:00    1
2015-10-03 12:00:00    1
2018-07-04 03:00:00    1
2017-02-15 22:00:00    1
2016-05-07 23:00:00    1
2018-05-08 18:00:00    1
2017-05-24 03:00:00    1
2012-10-29 03:00:00    1
2016-10-09 17:00:00    1
2013-11-23 06:00:00    1
2017-06-23 07:00:00    1
                      ..
2013-09-26 23:00:00    1
2014-01-18 04:00:00    1
2016-11-25 20:00:00    1
2012-11-27 04:00:00    1
2014-01-02 11:00:00    1
2017-10-08 21:00:00    1
2017-03-18 21:00:00    1
2014-02-04 13:00:00    1
2018-07-25 21:00:00    1


There are still a few duplicates left.

In [22]:
df[df['date_time'] == '2016-09-25 20:00:00']

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2,temp_avg,cloud_cover_avg
37901,None,288.03,0.25,0.0,80,Rain,light rain,2016-09-25 20:00:00,2710,2016-09-25,1.0,Rain,288.03,80.0
37902,None,287.73,0.00,0.0,48,Clouds,scattered clouds,2016-09-25 20:00:00,2710,2016-09-25,1.0,Rain,287.73,48.0


In [23]:
#do the same with rain_1h
df.reset_index(inplace=True)
df.drop(labels = 'index', axis=1, inplace=True)

df['rain_1h_avg'] = np.nan
df.loc[0, 'rain_1h_avg'] = df.loc[0, 'rain_1h']
average_duplicates(df, df['date_time'], df['rain_1h_avg'], df['rain_1h'])
df['rain_1h_avg'].fillna(value=df['rain_1h'], inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,weather_main_mark,weather_main_2,temp_avg,cloud_cover_avg,rain_1h_avg
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02,0.0,Clouds,288.28,40.0,0.0
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02,0.0,Clouds,289.36,75.0,0.0
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02,0.0,Clouds,289.58,90.0,0.0
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02,0.0,Clouds,290.13,90.0,0.0
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02,0.0,Clouds,291.14,75.0,0.0


In [24]:
df.drop_duplicates(['holiday', 'snow_1h', 'weather_main_2', 'date_time', 'traffic_volume'], inplace=True)
len(df)

40575

Length recorded at 40575 rows.

In [25]:
df['date_time'].value_counts()

2015-09-21 23:00:00    1
2014-08-03 13:00:00    1
2012-10-29 03:00:00    1
2018-09-30 06:00:00    1
2016-04-11 02:00:00    1
2013-12-29 02:00:00    1
2017-04-07 06:00:00    1
2017-05-08 09:00:00    1
2018-01-05 13:00:00    1
2013-06-12 13:00:00    1
2018-07-15 03:00:00    1
2018-05-08 18:00:00    1
2016-12-22 01:00:00    1
2013-06-07 07:00:00    1
2015-10-03 12:00:00    1
2018-07-04 03:00:00    1
2017-02-15 22:00:00    1
2016-05-07 23:00:00    1
2017-05-24 03:00:00    1
2015-07-06 15:00:00    1
2015-09-28 23:00:00    1
2017-09-10 11:00:00    1
2017-12-15 23:00:00    1
2017-04-18 07:00:00    1
2013-11-14 07:00:00    1
2016-11-25 13:00:00    1
2016-03-06 06:00:00    1
2013-11-23 06:00:00    1
2017-06-23 07:00:00    1
2017-09-12 12:00:00    1
                      ..
2015-12-07 17:00:00    1
2016-01-08 23:00:00    1
2013-05-29 09:00:00    1
2013-09-26 23:00:00    1
2017-06-22 04:00:00    1
2014-01-18 04:00:00    1
2012-11-27 04:00:00    1
2014-01-02 11:00:00    1
2017-10-08 21:00:00    1


All duplicates are gone!

In [26]:
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

Now that each timestamp has only one row, we can get to other feature engineering that is needed to predict traffic volume. New features to add:
- Dummy variables for each of the main weather types
- Dummy variables for the time of day
- Dummy variables for the day of the week (e.g., Tuesday)

In [27]:
#more feature engineering
df['day_of_week'] = df['date_time'].dt.weekday
day_of_week_dict = {0:'Sunday', 1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday'}
df['day_of_week'].replace(to_replace=day_of_week_dict, inplace=True)

df['time'] = df['date_time'].dt.time
df['time'] = df['time'].astype(str)

day_of_week_dummies = pd.get_dummies(df['day_of_week'])
day_of_week_dummies['date_time'] = df['date_time']
weather_dummies = pd.get_dummies(df['weather_main_2'])
weather_dummies['date_time'] = df['date_time']
time_dummies = pd.get_dummies(df['time'])
time_dummies['date_time'] = df['date_time']
    
weather_dummies = day_of_week_dummies.merge(weather_dummies, how='inner', on='date_time')
all_dummies = time_dummies.merge(weather_dummies, how='inner', on='date_time')

df2 = all_dummies = df.merge(all_dummies, how='inner', on='date_time')

In [28]:
df2.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume,date,...,Fog,Haze,Mist,Misty Haze,Mixed Precipitation,Rain,Smoke,Snow,Squall,Thunderstorm
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545,2012-10-02,...,0,0,0,0,0,0,0,0,0,0
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516,2012-10-02,...,0,0,0,0,0,0,0,0,0,0
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767,2012-10-02,...,0,0,0,0,0,0,0,0,0,0
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026,2012-10-02,...,0,0,0,0,0,0,0,0,0,0
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918,2012-10-02,...,0,0,0,0,0,0,0,0,0,0


We need to exclude one dummy variable in each category to have a baseline. The baseline here will be clear weather on Sunday at midnight.

In [29]:
indepvars_all = df2[['cloud_cover_avg', 'temp_avg', 'rain_1h_avg', 'snow_1h', 
                'Monday' ,'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
                '01:00:00', '02:00:00', '03:00:00', '04:00:00', '05:00:00', '06:00:00', '07:00:00','08:00:00',
                '09:00:00', '10:00:00', '11:00:00', '12:00:00', '13:00:00', '14:00:00', '15:00:00', '16:00:00', '17:00:00',
                '18:00:00', '19:00:00', '20:00:00', '21:00:00', '22:00:00', '23:00:00', 'Clouds', 'Rain', 'Mist',
               'Snow', 'Thunderstorm' ,'Mixed Precipitation', 'Drizzle', 'Haze', 'Fog', 'Misty Haze', 'Smoke', 'Squall']]
depvar = df2['traffic_volume']

Now we are ready to train-test-split.

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indepvars_all, depvar, random_state = 0)

I try linear regression first. More models to come.

In [31]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train, y_train)
[linreg.score(X_train, y_train), linreg.score(X_test, y_test)]

[0.8371964268926025, 0.8322883462642576]

In [32]:
import statsmodels.api as sm
regressors_all = sm.add_constant(indepvars_all)
est = sm.OLS(depvar, regressors_all)
est2 = est.fit()
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         traffic_volume   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     4593.
Date:                Mon, 29 Jul 2019   Prob (F-statistic):               0.00
Time:                        21:56:59   Log-Likelihood:            -3.2898e+05
No. Observations:               40575   AIC:                         6.581e+05
Df Residuals:                   40529   BIC:                         6.585e+05
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                -631.0955     90.982     -6.937      0.000    -809.421    -452.770
cloud_cover_avg        -0.6436      0.170     -3.788      0.000      -0.977      -0.311
temp_avg                5.4698      0.314     17.436      0.000       4.855       6.085
rain_1h_avg            -0.0012      0.082     -0.015      0.988      -0.162       0.159
snow_1h             -1096.8814    705.641     -1.554      0.120   -2479.954     286.191
Monday                222.7244     15.006     14.843      0.000     193.313     252.136
Tuesday               282.4218     14.946     18.897      0.000     253.128     311.716
Wednesday             331.6318     14.948     22.185      0.000     302.333     360.931
Thursday              360.8937     14.900     24.221      0.000     331.690     390.098
Friday               -495.8657     14.966    -33.133      0.000    -525.199    -466.532
Saturday             -905.4027     14.945    -60.583      0.000    -934.695    -876.110
01:00:00             -313.1456     27.451    -11.408      0.000    -366.950    -259.342
02:00:00             -446.3595     27.511    -16.225      0.000    -500.283    -392.436
03:00:00             -462.7863     27.636    -16.746      0.000    -516.954    -408.619
04:00:00             -125.4021     27.504     -4.559      0.000    -179.310     -71.494
05:00:00             1280.1496     27.585     46.408      0.000    1226.083    1334.216
06:00:00             3344.2506     27.598    121.176      0.000    3290.157    3398.344
07:00:00             3948.1777     27.655    142.766      0.000    3893.974    4002.382
08:00:00             3770.9692     27.572    136.770      0.000    3716.928    3825.010
09:00:00             3560.1603     27.767    128.218      0.000    3505.737    3614.583
10:00:00             3364.9591     27.497    122.376      0.000    3311.065    3418.854
11:00:00             3646.2190     27.681    131.723      0.000    3591.964    3700.474
12:00:00             3883.2181     27.600    140.698      0.000    3829.122    3937.314
13:00:00             3886.0613     27.698    140.301      0.000    3831.772    3940.350
14:00:00             4090.5274     27.630    148.044      0.000    4036.371    4144.684
15:00:00             4403.4789     27.762    158.615      0.000    4349.065    4457.893
16:00:00             4839.1767     27.581    175.456      0.000    4785.118    4893.235
17:00:00             4486.6370     27.764    161.599      0.000    4432.219    4541.055
18:00:00             3436.2887     27.659    124.236      0.000    3382.076    3490.502
19:00:00             2447.0294     27.632     88.557      0.000    2392.870    2501.189
20:00:00             2011.0732     27.508     73.109      0.000    1957.157    2064.989
21:00:00             1845.3105     27.503     67.095      0.000    1791.404    1899.217
22:

What happens if we exclude time-of-day dummy variables?

In [35]:
indepvars_ex_time = df2[['cloud_cover_avg', 'temp_avg', 'rain_1h_avg', 'snow_1h', 
                'Monday' ,'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Clouds', 'Rain', 'Mist',
               'Snow', 'Thunderstorm' ,'Mixed Precipitation', 'Drizzle', 'Haze', 'Fog', 'Misty Haze', 'Smoke', 'Squall']]
X_train_ex_time, X_test_ex_time, y_train_ex_time, y_test_ex_time = train_test_split(indepvars_ex_time, depvar, random_state = 0)
linreg_ex_time = LinearRegression().fit(X_train_ex_time, y_train_ex_time)
[linreg_ex_time.score(X_train_ex_time, y_train_ex_time), linreg_ex_time.score(X_test_ex_time, y_test_ex_time)]

[0.09208057968284555, 0.09005668595637828]

In [36]:
regressors_ex_time = sm.add_constant(indepvars_ex_time)
est_ex_time = sm.OLS(depvar, regressors_ex_time)
est2_ex_time = est_ex_time.fit()
est2_ex_time.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         traffic_volume   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     186.2
Date:                Mon, 29 Jul 2019   Prob (F-statistic):               0.00
Time:                        22:15:13   Log-Likelihood:            -3.6372e+05
No. Observations:               40575   AIC:                         7.275e+05
Df Residuals:                   40552   BIC:                         7.277e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -2905.1041    205.949    -14.106      0.000   -3308.769   -2501.439
cloud_cover_avg         3.4162      0.398      8.575      0.000       2.635       4.197
temp_avg               21.2219      0.723     29.369      0.000      19.806      22.638
rain_1h_avg             0.1772      0.192      0.921      0.357      -0.200       0.555
snow_1h              -692.1969   1660.123     -0.417      0.677   -3946.076    2561.682
Monday                237.4013     35.310      6.723      0.000     168.194     306.609
Tuesday               317.0345     35.167      9.015      0.000     248.106     385.963
Wednesday             349.6832     35.174      9.941      0.000     280.741     418.625
Thursday              386.2678     35.061     11.017      0.000     317.548     454.987
Friday               -456.6119     35.215    -12.967      0.000    -525.633    -387.590
Saturday             -867.2059     35.166    -24.660      0.000    -936.132    -798.280
Clouds                330.0112     33.530      9.842      0.000     264.291     395.731
Rain                  -42.1953     44.939     -0.939      0.348    -130.277      45.887
Mist                 -336.7875     45.758     -7.360      0.000    -426.474    -247.101
Snow                  -49.6715     54.169     -0.917      0.359    -155.844      56.501
Thunderstorm         -472.3047     73.239     -6.449      0.000    -615.854    -328.755
Mixed Precipitation   -71.8025     91.864     -0.782      0.434    -251.858     108.253
Drizzle              -157.9063     77.157     -2.047      0.041    -309.135      -6.678
Haze                  496.0838     84.856      5.846      0.000     329.765     662.403
Fog                  -608.4110     78.191     -7.781      0.000    -761.668    -455.154
Misty Haze            209.4581    101.874      2.056      0.040       9.783     409.133
Smoke                 -95.0403    447.001     -0.213      0.832    -971.172     781.092
Squall              -2096.8825   1892.495     -1.108      0.268   -5806.216    1612.451
==============================================================================
Omnibus:                    15959.118   Durbin-Watson:                   0.348
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2350.162
Skew:                          -0.226   Prob(JB):                         0.00
Kurtosis:                       1.911   Cond. No.                     5.74e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.74e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

It looks like the time of day explains most of the variation in traffic.